# HW3: Синтаксический анализ, коллокации

In [2]:
import pandas as pd
import numpy as np

import pymorphy2
import xml.etree.ElementTree as etree
import opencorpora
from pymystem3 import Mystem

morph = pymorphy2.MorphAnalyzer()

Проверим работу pymorphy2

In [3]:
morph.parse('матери')

[Parse(word='матери', tag=OpencorporaTag('NOUN,anim,femn sing,gent'), normal_form='матерь', score=0.2, methods_stack=((<DictionaryAnalyzer>, 'матери', 379, 1),)),
 Parse(word='матери', tag=OpencorporaTag('NOUN,anim,femn sing,gent'), normal_form='мать', score=0.2, methods_stack=((<DictionaryAnalyzer>, 'матери', 1917, 1),)),
 Parse(word='матери', tag=OpencorporaTag('NOUN,anim,femn sing,datv'), normal_form='матерь', score=0.1, methods_stack=((<DictionaryAnalyzer>, 'матери', 379, 2),)),
 Parse(word='матери', tag=OpencorporaTag('NOUN,anim,femn sing,loct'), normal_form='матерь', score=0.1, methods_stack=((<DictionaryAnalyzer>, 'матери', 379, 5),)),
 Parse(word='матери', tag=OpencorporaTag('NOUN,anim,femn plur,nomn'), normal_form='матерь', score=0.1, methods_stack=((<DictionaryAnalyzer>, 'матери', 379, 6),)),
 Parse(word='матери', tag=OpencorporaTag('VERB,impf,tran sing,impr,excl'), normal_form='материть', score=0.1, methods_stack=((<DictionaryAnalyzer>, 'матери', 463, 11),)),
 Parse(word='ма

In [35]:
def read_txt(file):
    ''' 
    Функция считывает текст из файла .txt, разбивает на токены, проводит морфологический анализ каждого токена, возвращает 
    текст и список токенов в формате pymorphy2. 
    '''
    text = open(file, 'r').read()
    l = Mystem().analyze(text)
    tokens = []
    for t in l:
        try:
            tokens.append((t['text'],t['analysis']))
        except KeyError:
            pass    
    words = [word[0] for word in tokens]
    morph_w = [morph.parse(w)[0] for w in words]
    return text, morph_w

In [61]:
def find_AN(tokens_parsed):
    '''
     принимает список токенов в формате pymorphy, возвращает список словосочетаний формата прилагательное+существительное 
     и существительное+прилагательное. Словосочетание считается согласованным, если у прилагательного и существительного 
     совпадают род, число и падеж. Числительные и местоименные прилагательные так же учитываются.
    '''
    AN_list = []
    for i in range(len(tokens_parsed)):
        if tokens_parsed[i].tag.POS == 'ADJF':
            for j in range(i+1,i+4):
                if (tokens_parsed[j].tag.POS == 'NOUN' and 
                    tokens_parsed[i].tag.gender == tokens_parsed[j].tag.gender and 
                    tokens_parsed[i].tag.number == tokens_parsed[j].tag.number and
                    tokens_parsed[i].tag.case == tokens_parsed[j].tag.case):
                    AN_list.append(tokens_parsed[i].word+' '+tokens_parsed[j].word)
            for j in range(i-1,i):
                if (tokens_parsed[j].tag.POS == 'NOUN' and 
                    tokens_parsed[i].tag.gender == tokens_parsed[j].tag.gender and 
                    tokens_parsed[i].tag.number == tokens_parsed[j].tag.number and
                    tokens_parsed[i].tag.case == tokens_parsed[j].tag.case):
                    AN_list.append(tokens_parsed[j].word+' '+tokens_parsed[i].word)

    return AN_list

In [71]:
def find_VN_tran(tokens):
    '''
    принимает список токенов в формате pymorphy, возвращает список словосочетаний формата 
    переходный глагол+NP(вида прилагательные+существительное в винительном падеже)
    '''
    AN_groups = dict()
    for i in range(len(tokens)):
        if tokens[i].tag.POS == 'ADJF':
            for j in range(i+1,i+4):
                if (tokens[j].tag.POS == 'NOUN' and 
                    tokens[i].tag.gender == tokens[j].tag.gender and 
                    tokens[i].tag.number == tokens[j].tag.number and
                    tokens[i].tag.case == tokens[j].tag.case):
                        AN_groups[j]=tokens[i].word+' '+AN_groups.get(j,'')
                    
#            for j in range(i-1,i):
#                if (tokens_parsed[j].tag.POS == 'NOUN' and 
#                    tokens_parsed[i].tag.gender == tokens_parsed[j].tag.gender and 
#                    tokens_parsed[i].tag.number == tokens_parsed[j].tag.number and
#                    tokens_parsed[i].tag.case == tokens_parsed[j].tag.case):
#                    AN_groups[j]=AN_groups[j].get('')+' '+tokens_parsed[i].word

#                    AN_list.append(tokens_parsed[j].word+' '+tokens_parsed[i].word)
    VN_list = []
    for i in range(len(tokens)):
        if tokens[i].tag.POS == 'VERB' and tokens[i].tag.transitivity == 'tran':
            for j in range(i+1,i+3):
                if (tokens[j].tag.POS == 'NOUN' and tokens[j].tag.case == 'accs'):
                    VN_list.append(tokens[i].word+' '+AN_groups.get(j,'')+tokens[j].word)
    return VN_list

In [79]:
def find_AV(tokens):
    VN_list = []
    VN_groups = {}
    for i in range(len(tokens)):
        if tokens[i].tag.POS == 'ADVB':
            for j in range(i+1,i+3):
                if tokens[j].tag.POS == 'VERB':
                    VN_list.append(tokens[i].word+' '+VN_groups.get(j,'')+tokens[j].word)
                    VN_groups[j]=tokens[i].word+' '+VN_groups.get(j,'')
            for j in range(i-2,i):
                if tokens[j].tag.POS == 'VERB':
                    VN_list.append(tokens[j].word+' '+VN_groups.get(j,'')+tokens[i].word)
                    VN_groups[j]=tokens[i].word+' '+VN_groups.get(j,'')


    return VN_list   

## Репортаж Медузы

In [36]:
text, toks = read_txt('text.txt')

In [37]:
print(text)

На выборах 9 сентября 2018 года Якутск стал единственной региональной столицей, где мэром был избран представитель оппозиции. 48-летняя Сардана Авксентьева обошла единоросса Александра Саввинова, поддержанного губернатором, и стала первой женщиной-градоначальником в истории города. После избрания Авксентьева развернула бурную деятельность, направленную на исполнение «народного заказа»: принялась сокращать расходы на городской аппарат; увольнять сомнительных чиновников и подрядчиков; распродавать записанные на мэрию дорогие машины. В результате Авксентьева стала общероссийской знаменитостью — а успехами якутской мэрии в интернете даже заинтересовались в администрации президента. Спецкор «Медузы» Таисия Бекбулатова отправилась в Якутск, чтобы понять, как Авксентьевой удалось победить и какое будущее ее ждет.


3 августа 2018 года — за пять недель до выборов главы Якутска — Сардане Авксентьевой позвонили. Голос в телефонной трубке объявил: «Ну все, мать, ты мэр».

Звонил якутский бизнесме

In [80]:
find_AV(toks)

['где был',
 'уже участвовал',
 'быстро повысят',
 'действительно возглавил',
 'было по-другому',
 'сейчас говорит',
 'целенаправленно шёл',
 'снова отказался',
 'уже сидел',
 'прямо бежит',
 'одновременно снял',
 'отзываем одновременно',
 'была уже',
 'говорит естественно',
 'прекрасно понимал',
 'будет выше',
 'далее рассказывает',
 'естественно был',
 'хорошо запомнили',
 'появились вынужденно',
 'уже имела',
 'вышла замуж',
 'где был',
 'уже участвовал',
 'быстро повысят',
 'действительно возглавил',
 'было по-другому',
 'сейчас говорит',
 'целенаправленно шёл',
 'снова отказался',
 'уже сидел',
 'прямо бежит',
 'одновременно снял',
 'отзываем одновременно',
 'была уже',
 'говорит естественно',
 'прекрасно понимал',
 'будет выше',
 'далее рассказывает',
 'естественно был',
 'хорошо запомнили',
 'появились вынужденно',
 'уже имела',
 'вышла замуж',
 'тоже встречался',
 'активно тоже встречался',
 'часто приходили',
 'вечером ездили',
 'вдвоём спали',
 'ездили вечером вдвоём',
 'испр

In [72]:
find_VN_tran(toks)

['развернула бурную деятельность',
 'занял место',
 'возглавил якутию',
 'подвело разгар',
 'развернула бурную деятельность',
 'занял место',
 'возглавил якутию',
 'подвело разгар',
 'разыграл национальную карту',
 'делала упор',
 'подчёркивает проигрыш',
 'одобрил пенсионную реформу',
 'поддержала инициативу',
 'накрыла россию',
 'выполняла распоряжение',
 'подписала распоряжение',
 'вызвали одобрение',
 'любит время',
 'устраивает раз',
 'устанавливают большую палатку',
 'покинул пост',
 'поменял отношение',
 'дал интервью',
 'сказал волю',
 'вызывает волну',
 'отменила дорогую инаугурацию',
 'опередила время']

In [58]:
find_AN(toks)

['единственной столицей',
 'бурную деятельность',
 'городской аппарат',
 'якутской мэрии',
 'какое будущее',
 'якутский бизнесмен',
 'телефонного разговора',
 'летней авксентьевой',
 'партийное руководство',
 'фёдоров который',
 'свою кандидатуру',
 'этой подготовки',
 'московский парень',
 'фёдоров один',
 'дальнейшем развитии',
 'значительную роль',
 'личная мотивация',
 'моя планка',
 'сам фёдоров',
 'того заседания',
 'самого избиркома',
 'стройная женщина',
 'темноволосая женщина',
 'фирменным знаком',
 'всю жизнь',
 'муниципального управления',
 'единой россии',
 'единую россию',
 'единственной столицей',
 'бурную деятельность',
 'городской аппарат',
 'якутской мэрии',
 'какое будущее',
 'якутский бизнесмен',
 'телефонного разговора',
 'летней авксентьевой',
 'партийное руководство',
 'фёдоров который',
 'свою кандидатуру',
 'этой подготовки',
 'московский парень',
 'фёдоров один',
 'дальнейшем развитии',
 'значительную роль',
 'личная мотивация',
 'моя планка',
 'сам фёдоров',
 

In [42]:
toks

[Parse(word='на', tag=OpencorporaTag('PREP'), normal_form='на', score=0.99931, methods_stack=((<DictionaryAnalyzer>, 'на', 24, 0),)),
 Parse(word='выборах', tag=OpencorporaTag('NOUN,inan,GNdr,Pltm plur,loct'), normal_form='выборы', score=0.666666, methods_stack=((<DictionaryAnalyzer>, 'выборах', 133, 5),)),
 Parse(word='сентября', tag=OpencorporaTag('NOUN,inan,masc sing,gent'), normal_form='сентябрь', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'сентября', 190, 1),)),
 Parse(word='года', tag=OpencorporaTag('NOUN,inan,masc sing,gent'), normal_form='год', score=0.999139, methods_stack=((<DictionaryAnalyzer>, 'года', 1168, 1),)),
 Parse(word='якутск', tag=OpencorporaTag('NOUN,inan,masc,Geox sing,nomn'), normal_form='якутск', score=0.5, methods_stack=((<DictionaryAnalyzer>, 'якутск', 73, 0),)),
 Parse(word='стал', tag=OpencorporaTag('VERB,perf,intr masc,sing,past,indc'), normal_form='стать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'стал', 904, 1),)),
 Parse(word='единственной

## Научпоп

In [81]:
text2, toks2 = read_txt('text_motova.txt')

In [82]:
find_AN(toks2)

['научный журналист',
 'калифорнийском университете',
 'доказательной диетологии',
 'доказательной лекции',
 'новый формат',
 'эту книгу',
 'одном флаконе',
 'доказательная диетология',
 'пищеварительная система',
 'сладкий горький',
 'солёный горький',
 'кислый горький',
 'китайская гастрономия',
 'питательную еду',
 'полезную еду',
 'материнское молоко',
 'электролитного баланса',
 'основным материалом',
 'белком основным',
 'концентрированном виде',
 'следующего приёма',
 'ежедневную еду',
 'еда незрелая',
 'многодетная мать',
 'том числе',
 'эта капуста',
 'вкусовая чувствительность',
 'вкусовая стройность',
 'эталонное вещество',
 'химическое вещество',
 'пищевым красителем',
 'поддержаниестабильного веса',
 'наше поведение',
 'пищевое поведение',
 'красивой подаче',
 'меньшем количестве',
 'французский парадокс',
 'лучшая приправа',
 'большая часть',
 'шоколадному следу',
 'каждый вид',
 'привычную еду',
 'острого перца',
 'жгучей боли',
 'такую еду',
 'острая еда']

In [83]:
find_VN_tran(toks2)

['ведёт блог',
 'чувствуем вкус',
 'регулируют вес',
 'написала эту книгу',
 'дают энергию',
 'содержит лактозу',
 'предложите овощи',
 'отвергает такую еду']

In [84]:
find_AV(toks2)

['постоянно учится',
 'хорошо готовит',
 'почему толстеют',
 'останется осторожно',
 'буквально заставляет',
 'улавливает буквально',
 'гораздо уловила',
 'развивалась эволюционно',
 'получим достаточно',
 'сообщает насколько',
 'поэтому отказываются',
 'рождаются редко',
 'попробуйте сегодня',
 'увлекайтесь избыточно',
 'много пытайтесь']

## Специализированный текст о разработке

In [99]:
text3, toks3 = read_txt('text_taxi.txt')

In [100]:
find_VN_tran(toks3)

['нажимает кнопку',
 'ранжировали водителей',
 'используем множество',
 'забили пул',
 'добавили отправку',
 'нашли заказ',
 'играют роль',
 'сделали возможность',
 'сохранили логику',
 'добавили возможность',
 'нажмёт кнопку',
 'требует множество']

In [101]:
find_AV(toks3)

['быстро приезжала',
 'сегодня расскажу',
 'выбираем наиболее',
 'поэтому поступает',
 'сначала определяет',
 'затем уточняется',
 'ранжировали уже',
 'могут вообще',
 'годится максимально',
 'лежит хорошо',
 'вкратце суть',
 'совсем вписывалось',
 'пришлось немного',
 'сразу договорились',
 'индивидуально попадали',
 'поэтому добавили',
 'обрабатывались параллельно',
 'обрабатывались параллельно попутно',
 'корректно работает',
 'корректно появился',
 'уже работало',
 'почти уже работало',
 'работало почти уже почти',
 'работало почти почти уже потому',
 'извне могли',
 'контролировались извне',
 'тут играют',
 'происходит сразу',
 'компенсируется более',
 'далее последует']

In [102]:
find_AN(toks3)

['холостого пробега',
 'подходящего водителя',
 'общая архитектура',
 'конечным автоматом',
 'жадный подход',
 'жадный подход',
 'таком подходе',
 'таком этапе',
 'локальном геоиндексе',
 'прямому радиусу',
 'этой информации',
 'лучший вариант',
 'эта логика',
 'такое назначение',
 'жадное назначение',
 'буферный подход',
 'жадном подходе',
 'ближайшего водителя',
 'повышенном спросе',
 'жадный подход',
 'самые часы',
 'буферное назначение',
 'известная задача',
 'комбинаторной оптимизации',
 'любую задачу',
 'один исполнитель',
 'одну работу',
 'такой задачи',
 'наша стоимость',
 'одной стороны',
 'такую задачу',
 'венгерский алгоритм',
 'жадном подходе',
 'отдельном микросервисе',
 'жадном подходе',
 'одной машины',
 'разумного количества',
 'жадное назначение',
 'новом сервисе',
 'наш сервис',
 'ту сторону',
 'конечным автоматом',
 'буферном диспатче',
 'один способ',
 'буферном поиске',
 'подходящему водителю',
 'непростая задача']